<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Query for APEX Data**

This notebook provides an example of how to query the ESO Science Archive for **APEX data**. 

APEX data are available within the archive and can be queried using any of the standard astroquery methods:  
- `eso.query_survey` for **reduced data**  
- `eso.query_main` for **raw data**  
- `eso.query_instrument` for **raw data**

In addition, there is the option to use `eso.query_apex_quicklooks`, which is a dedicated query for retrieving **APEX Quick Look products**.

The workflow for APEX data is somewhat different from that of other ESO-operated instruments and telescopes. The **APEX Quick Looks** aim to address some of these differences by providing users with `.tar` files that include additional data outputs — such as diagnostic plots, observing logs, and (in some cases) a reduced version of the dataset. These `.tar` files also include `class` (`.apex`) files that allow users to **re-run the data reduction**, if desired. This is often more useful than working directly with the `.fits` files available in the archive and can be **particularly helpful for heterodyne datasets**. For heterodyne observations, the calibrated `.class` file is typically included, while the corresponding uncalibrated `.fits` files are usually **not needed** and cannot be processed with standard reduction software.

The APEX Science Archive provides access to raw observations and associated data products for observations performed with APEX since **July 11, 2005**. The Quick Look products are designed to help users preview and assess the data quality, providing useful summaries that complement the raw observations.

This notebook will guide you through a step-by-step workflow to **identify, query, and download** APEX Quick Look data products from the ESO Science Archive.

Here, we focus on APEX observations from the [**ALCOHOLS survey**](https://www.eso.org/rm/api/v1/public/releaseDescriptions/199) (12CO(3–2) line emission in the Milky Way) available in the ESO Archive. We will first search for the **reduced ALCOHOLS data products**, then identify the corresponding raw datasets using the **instrument-specific query**. Once the raw data is identified, we can use it to locate and retrieve the **associated APEX Quick Look products**. We follow this workflow because APEX Quick Looks typically require knowledge of the **APEX proposal ID**, which may not always be known in advance — this is **not** the same as the ESO programme ID. If you know your APEX proposal ID (for example, if you are querying your own data), you can search for APEX Quick Look files directly.

Note that this workflow is **not specific to the ALCOHOLS survey** and can be used to query APEX Quick Look products for any project.

<hr style="border:2px solid #0281c9"> </hr>

# **Importing and basic usage of astroquery.eso**

In [1]:
import astroquery # import astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

astroquery version: 0.4.11.dev10245


In [2]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [3]:
eso = Eso() # create an instance of the ESO class 

# **Query for Reduced APEX data**

We first query for the reduced data from the **ALCOHOLS** survey, and retrieve the proposal ID (if you know your proposal ID then this step can be skipped). 

Note that multiple proposal IDs may be returned, which would require minor changes to the script to loop through the project.

In [4]:
table_reduced = eso.query_surveys("ALCOHOLS") # query the ESO archive for the ALCOHOLS survey
proposal_id = list(set(table_reduced['proposal_id'])) # extract unique proposal IDs from the query result

# Check if we have a single proposal ID or multiple
if len(proposal_id) == 1:
    proposal_id = proposal_id[0]
else:
    print("Warning: Multiple proposal IDs found...")

proposal_id = proposal_id.split('(')[0] # extract the first part of the proposal ID before any parentheses (i.e. the run ID)
print(f"Proposal ID: {proposal_id}")

Proposal ID: 094.C-0935


# **Query for Raw APEX data**

As always, it is good practice to check the available columns to search in the instrument-specific query.

In [5]:
eso.query_instrument("APEX", help=True) # get help on the query_instrument method

INFO: 
Columns present in the table ist.APEX:
   column_name    datatype    xtype     unit
----------------- -------- ----------- -----
   access_estsize     long             kbyte
       access_url     char                  
             bwid    float               GHz
         channels      int                  
     datalink_url     char                  
         date_obs     char                  
              dec   double               deg
           dp_cat     char                  
            dp_id     char                  
          dp_tech     char                  
          dp_type     char                  
          ecl_lat   double               deg
          ecl_lon   double               deg
        exp_start     char   timestamp      
         exposure    float                 s
          exptime    float                 s
             febe     char                  
             freq    float               GHz
          freqres    float                  
         

We now query for raw data from the APEX instrument, using the proposal ID we retrieved from the previous query.

In this case, we know there is only **one** APEX proposal ID, but if there were multiple IDs, we would need to loop through them.

> **Note:** In the `eso.query_surveys` query, the `"proposal_id"` column refers to the **ESO programme ID**. In contrast, in an APEX-specific query using `eso.query_instrument`, the `"prog_id"` column also refers to the **ESO programme ID**, **not** the **APEX proposal ID**. The APEX proposal ID is instead found in the `"project_id"` column in the `eso.query_instrument` query — this is the value used to identify APEX Quick Look products.

In [6]:
table_raw = eso.query_instrument("APEX", column_filters={"prog_id": f"like '{proposal_id}%'"}) # query the APEX instrument for data related to the proposal ID
project_id = list(set(table_raw["project_id"])) # extract unique project IDs from the raw data query
project_id = project_id[0] # Assuming we only have one project ID -- see above check... 
print(f"Project ID: {project_id}")

Project ID: E-094.C-0935A-2014


**Bonus:** Just for fun, we can also print the column descriptions of the instrument-specific raw query table...

In [10]:
print("Column Descriptions:\n" + "-" * 25)
for colname in table_raw.colnames:
    desc = table_raw[colname].info.description
    desc_str = desc if desc is not None else "[no description]"
    print(f"{colname:>25}: {desc_str}")

Column Descriptions:
-------------------------
                   object: Identifier or name of the target observed. It is either the target designation (as given by the astronomer) for science exposures, or the exposure type for non-science frames (e.g. FLAT,LAMP). Originating from FITS keyword: OBJECT.
                       ra: Right Ascension of the pointing (FK5). Expressed in degrees. Originating from FITS keyword: RA.
                      dec: Declination of the pointing (FK5). Expressed in degrees. Originating from FITS keyword: DEC.
                    dp_id: Primary unique identifier for the ESO archive file.
                 date_obs: A string (varchar, not  a datetime) representing the date and time of exposure start (International Atomic Time - TAI). It can be used to match (also using SQL wild cards) the string expressed as ISO 8601 compliant date-time format 'YYYY-MM-DDTHH:MM:SS(.sss..)', with a 'T' separating the date and time (e.g., 2000-01-01T00:00:00). Not being a d

# **Query for APEX Quick Look products**

As always, we can check the available columns to search in the query.

In [7]:
eso.query_apex_quicklooks(help=True) # get help on the query_apex_quicklooks method

INFO: 
Columns present in the table ist.apex_quicklooks:
  column_name   datatype   xtype    unit
--------------- -------- --------- -----
 access_estsize     long           kbyte
     access_url     char                
     instrument     char                
instrument_type     char                
        partner     char                
         pi_coi     char                
        prog_id     char                
     prog_title     char                
      prog_type     char                
     project_id     char                
   quicklook_id     char                
   release_date     char timestamp      

Number of records present in the table ist.apex_quicklooks:
282296
 [astroquery.eso.core]


We can now query for the APEX Quick Look products using the APEX proposal ID (``project_id``) we retrieved from the previous query.

In [8]:
table_quicklooks = eso.query_apex_quicklooks(project_id) # query the APEX quicklooks for the project ID
table_quicklooks[:3]  # print the quicklook table

access_estsize,access_url,instrument,instrument_type,partner,pi_coi,prog_id,prog_title,prog_type,project_id,quicklook_id,release_date
kbyte,,,,,,,,,,,
int64,object,object,object,object,object,object,object,object,object,object,object
846755,https://dataportal.eso.org/dataPortal/file/E-094.C-0935A.2014DEC10.TAR,APEXHET,Heterodyne,ESO,Stanke/ Arce/ Bally/ Bergman/ Carpenter/ Davis/ Dent/ Di Francesco/ Eisloeffel/ Froebrich/ Ginsburg/ Heyer/ Johnstone/ Mardones/ McCaughrean/ Megeath/ Smith/ Nakamura/ Reipurth/ Stutz/ Tatematsu/ Walker/ Williams/ Zinnecker,094.C-0935(A),The APEX Large CO Heterodyne Outflow Legacy Supercam survey of Orion (ALCOHOLS),Normal,E-094.C-0935A-2014,E-094.C-0935A.2014DEC10.TAR,2014-12-10T07:05:44.397Z
2320209,https://dataportal.eso.org/dataPortal/file/E-094.C-0935A.2014DEC12.TAR,APEXHET,Heterodyne,ESO,Stanke/ Arce/ Bally/ Bergman/ Carpenter/ Davis/ Dent/ Di Francesco/ Eisloeffel/ Froebrich/ Ginsburg/ Heyer/ Johnstone/ Mardones/ McCaughrean/ Megeath/ Smith/ Nakamura/ Reipurth/ Stutz/ Tatematsu/ Walker/ Williams/ Zinnecker,094.C-0935(A),The APEX Large CO Heterodyne Outflow Legacy Supercam survey of Orion (ALCOHOLS),Normal,E-094.C-0935A-2014,E-094.C-0935A.2014DEC12.TAR,2014-12-12T11:05:59.437Z
5445826,https://dataportal.eso.org/dataPortal/file/E-094.C-0935A.2014DEC13.TAR,APEXHET,Heterodyne,ESO,Stanke/ Arce/ Bally/ Bergman/ Carpenter/ Davis/ Dent/ Di Francesco/ Eisloeffel/ Froebrich/ Ginsburg/ Heyer/ Johnstone/ Mardones/ McCaughrean/ Megeath/ Smith/ Nakamura/ Reipurth/ Stutz/ Tatematsu/ Walker/ Williams/ Zinnecker,094.C-0935(A),The APEX Large CO Heterodyne Outflow Legacy Supercam survey of Orion (ALCOHOLS),Normal,E-094.C-0935A-2014,E-094.C-0935A.2014DEC13.TAR,2015-02-20T19:15:36.383Z


Again, lets print some information about the columns in the APEX Quick Look table...

In [11]:
print("Column Descriptions:\n" + "-" * 25)
for colname in table_quicklooks.colnames:
    desc = table_quicklooks[colname].info.description
    desc_str = desc if desc is not None else "[no description]"
    print(f"{colname:>25}: {desc_str}")

Column Descriptions:
-------------------------
           access_estsize: Estimated size of the APEX quick look TAR file in kilo bytes.
               access_url: A direct URL for downloading the quicklook product. If the file is under a proprietary period, access is restricted to authenticated users with the necessary permissions.
               instrument: Type of the instrument (APEXHET = heterodynes, APEXBOL =  bolometers). Originating from FITS keyword: INSTRUME.
          instrument_type: Type of the instrument used: Heterodyne or Bolometer.
                  partner: This field indicates which partner institute an APEX project belongs to, it can be: ESO, Max-Planck, Onsala, Chilean or Technical project.
                   pi_coi: Names of the Principal Investigator (PI) and Co-Investigator(s) associated with the observation.
                  prog_id: Identifier for the ESO observing program linked to the observation. Originating from FITS keyword: HIERARCH ESO OBS PROG ID.
    

As can be seen from the output above, there is one APEX Quick Look product available per UT date, per APEX proposal ID. 

Also note that the APEX Quick Look products are available in `.tar` (`.TAR`) format, which can be downloaded and extracted (see below). 

## **Download APEX Quick Look products**

Finally, we can download the APEX Quick Look products using the `eso.retrieve_data` function.

In [9]:
# eso.retrieve_data(table_quicklooks[0]["quicklook_id"])  # download the first quicklook data file